<a href="https://colab.research.google.com/github/mrm8488/shared_colab_notebooks/blob/master/RuPERTa_base_cased_finetuned_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to fine tune RuPERTa-base for NER downstream task

> Creator: [Manuel Romero](https://twitter.com/mrm8488)

In [2]:
!nvidia-smi

Thu May 14 15:57:48 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 26445 (delta 3), reused 4 (delta 0), pack-reused 26431
Receiving objects: 100% (26445/26445), 15.92 MiB | 27.63 MiB/s, done.
Resolving deltas: 100% (18408/18408), done.


In [4]:
!pip install -q ./transformers
!pip install -q tensorboardX
!pip install -q seqeval

     |████████████████████████████████| 3.8MB 3.4MB/s 
     |████████████████████████████████| 1.1MB 33.2MB/s 
     |████████████████████████████████| 890kB 35.3MB/s 
     |████████████████████████████████| 204kB 3.2MB/s 


In [0]:
!wget https://raw.githubusercontent.com/mrm8488/NER/master/train.txt
!wget https://raw.githubusercontent.com/mrm8488/NER/master/dev.txt
!wget https://raw.githubusercontent.com/mrm8488/NER/master/labels.txt

In [0]:
!mkdir ner_dataset

In [0]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

In [8]:
!python3 preprocess.py train.txt "mrm8488/RuPERTa-base" 256 > /content/ner_dataset/train.txt
!python3 preprocess.py dev.txt "mrm8488/RuPERTa-base" 256 > /content/ner_dataset/dev.txt

2020-05-14 16:57:56.444513: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Downloading: 100% 524/524 [00:00<00:00, 329kB/s]
Downloading: 100% 981k/981k [00:00<00:00, 5.60MB/s]
Downloading: 100% 523k/523k [00:00<00:00, 3.58MB/s]
Downloading: 100% 150/150 [00:00<00:00, 105kB/s]
Downloading: 100% 2.00/2.00 [00:00<00:00, 1.28kB/s]
2020-05-14 16:58:24.181450: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1


In [0]:
!pip install -q wandb

In [0]:
!wandb login

In [0]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir /content/runs

In [0]:
%env WANDB_PROJECT="fine-tune-RuPERTa-spanish-NER"

env: WANDB_PROJECT="fine-tune-RuPERTa-spanish-NER"


In [0]:
!python /content/transformers/examples/token-classification/run_ner.py --data_dir /content/ner_dataset \
--model_type roberta \
--labels /content/labels.txt \
--model_name_or_path mrm8488/RuPERTa-base \
--output_dir model_output \
--max_seq_length 256 \
--num_train_epochs 20 \
--per_gpu_train_batch_size 32 \
--overwrite_output_dir \
--save_steps 1000 \
--do_train \
--do_eval

In [0]:
!mkdir /content/RuPERTa-base-finetuned-ner

In [0]:
!mv /content/model_output/* /content/RuPERTa-base-finetuned-ner

In [0]:
!rm -rf /content/RuPERTa-base-finetuned-ner/checkpoint-*

In [0]:
!transformers-cli login

In [0]:
!transformers-cli upload ./RuPERTa-base-finetuned-ner

## Test the model

In [0]:
from transformers import pipeline

In [23]:
nlp_ner = pipeline(
    "ner",
    model="mrm8488/RuPERTa-base-finetuned-ner",
    tokenizer="mrm8488/RuPERTa-base-finetuned-ner")

text = 'Mis amigos están pensando viajar a Londres este verano.'

nlp_ner(text)

[{'entity': 'NUM', 'score': 0.2680954933166504, 'word': '<s>'},
 {'entity': 'NUM', 'score': 0.9989176988601685, 'word': 'ĠMis'},
 {'entity': 'PRON', 'score': 0.568749725818634, 'word': 'Ġamigos'},
 {'entity': 'AUX', 'score': 0.9526389241218567, 'word': 'ĠestÃ¡n'},
 {'entity': 'ADV', 'score': 0.7196545600891113, 'word': 'Ġpensando'},
 {'entity': 'VERB', 'score': 0.3267662823200226, 'word': 'Ġviajar'},
 {'entity': 'ADP', 'score': 0.9935892820358276, 'word': 'Ġa'},
 {'entity': 'NOUN', 'score': 0.4341692328453064, 'word': 'ĠLondres'},
 {'entity': 'DET', 'score': 0.8386265635490417, 'word': 'Ġeste'},
 {'entity': 'NOUN', 'score': 0.9109864234924316, 'word': 'Ġverano'},
 {'entity': 'PUNCT', 'score': 0.9993612170219421, 'word': '.'},
 {'entity': 'ADV', 'score': 0.23953089118003845, 'word': '</s>'}]